<a href="https://colab.research.google.com/github/Coden-inja/colab-pipeline-test/blob/main/abr_testing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# BLOCK 1: SETUP
print("⚙️ Installing ffmpeg-python...")
!pip install ffmpeg-python -q

import ffmpeg
import os
import sys
import json
import logging
import platform
import subprocess
import shutil
import glob

# Configure logging to show us everything
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(message)s')
logger = logging.getLogger(__name__)

print("✅ Environment Ready.")

⚙️ Installing ffmpeg-python...
✅ Environment Ready.


In [2]:
# BLOCK 2: DEFINE PROCESSOR
class ABRVideoProcessor:
    def __init__(self):
        # REFERENCE TIERS: The menu of options we can pick from
        self.REFERENCE_TIERS = [
            {"name": "8k",    "width": 7680, "height": 4320, "bitrate": "40000k"},
            {"name": "4k",    "width": 3840, "height": 2160, "bitrate": "18000k"},
            {"name": "1440p", "width": 2560, "height": 1440, "bitrate": "10000k"},
            {"name": "1080p", "width": 1920, "height": 1080, "bitrate": "5000k"},
            {"name": "720p",  "width": 1280, "height": 720,  "bitrate": "2500k"}
        ]

        # COLAB SAFETY PATCH: Force CPU mode to avoid "Ghost GPU" crash
        self.hardware_config = {'c:v': 'libx264', 'preset': 'ultrafast'}

    def _analyze_input(self, input_path):
        """Probes input video for exact dimensions"""
        try:
            probe = ffmpeg.probe(input_path)
            video_stream = next(s for s in probe['streams'] if s['codec_type'] == 'video')
            width = int(video_stream['width'])
            height = int(video_stream['height'])

            avg_frame_rate = video_stream.get('avg_frame_rate', '30/1')
            num, den = map(int, avg_frame_rate.split('/'))
            fps = num / den if den > 0 else 30

            return {"width": width, "height": height, "gop": int(round(fps * 2))}
        except Exception as e:
            logger.error(f"Probe failed: {e}")
            sys.exit(1)

    def _generate_ladder(self, input_w, input_h):
        """Decides which resolutions to generate"""
        ladder = []
        logger.info(f"🤔 DECISION TIME: Input is {input_w}x{input_h}")

        for tier in self.REFERENCE_TIERS:
            if tier['width'] <= input_w:
                logger.info(f"   ✅ Keeping {tier['name']} (Smaller than or equal to source)")
                ladder.append(tier)
            else:
                logger.info(f"   ❌ Skipping {tier['name']} (Too big for source)")

        # Fallback: If video is weirdly small (e.g. 480p), keep original
        if not ladder:
             ladder.append({"name": "original", "width": input_w, "height": input_h, "bitrate": "1000k"})

        return ladder

    def process(self, input_path, output_root):
        if not os.path.exists(input_path):
            return

        vid_id = os.path.splitext(os.path.basename(input_path))[0]
        base_dir = os.path.join(output_root, vid_id)
        if os.path.exists(base_dir): shutil.rmtree(base_dir)
        os.makedirs(base_dir, exist_ok=True)

        # 1. Analyze
        info = self._analyze_input(input_path)
        ladder = self._generate_ladder(info['width'], info['height'])

        master_playlist_content = "#EXTM3U\n#EXT-X-VERSION:3\n"

        # 2. Poster
        logger.info("🎨 Generating Poster...")
        poster_path = os.path.join(base_dir, 'poster.jpg')
        (
            ffmpeg.input(input_path)
            .filter('select', 'gte(n,0)')
            .output(poster_path, vframes=1, q=2, loglevel="error")
            .run(overwrite_output=True)
        )

        # 3. Transcode Ladder
        for variant in ladder:
            variant_name = variant['name']
            variant_dir = os.path.join(base_dir, variant_name)
            os.makedirs(variant_dir, exist_ok=True)
            output_m3u8 = os.path.join(variant_dir, 'index.m3u8')

            logger.info(f"🎬 Transcoding Variant: {variant_name}...")

            stream = ffmpeg.input(input_path)
            stream = ffmpeg.output(
                stream,
                output_m3u8,
                format='hls',
                hls_time=2,
                hls_list_size=0,
                hls_segment_filename=os.path.join(variant_dir, 'seg_%03d.ts'),
                vf=f"scale={variant['width']}:-2",
                **{'b:v': variant['bitrate'], 'maxrate': variant['bitrate']},
                **{'bufsize': str(int(variant['bitrate'].replace('k','')) * 2) + 'k'},
                g=info['gop'],
                keyint_min=info['gop'],
                sc_threshold=0,
                **{'c:a': 'aac', 'b:a': '128k'},
                **self.hardware_config
            )
            stream.run(overwrite_output=True, quiet=True)

            bandwidth = int(variant['bitrate'].replace('k', '')) * 1000
            master_playlist_content += (
                f"#EXT-X-STREAM-INF:BANDWIDTH={bandwidth},RESOLUTION={variant['width']}x{variant['height']}\n"
                f"{variant_name}/index.m3u8\n"
            )

        with open(os.path.join(base_dir, 'master.m3u8'), 'w') as f:
            f.write(master_playlist_content)

        # Standard Output (JSON for your API)
        print(json.dumps({
            "status": "success",
            "stream_url": f"/videos/{vid_id}/master.m3u8",
            "poster_url": f"/videos/{vid_id}/poster.jpg"
        }))

print("✅ Processor Class Loaded.")

✅ Processor Class Loaded.


In [4]:
# BLOCK 3 (DEBUG MODE): GENERATE MOCK 4K VIDEO
import subprocess
import os

print("🎥 Generating Mock 4K Source (3840x2160)...")

# We removed 'stderr=subprocess.DEVNULL' so we can see the error
result = subprocess.run([
    'ffmpeg', '-y', '-f', 'lavfi',
    '-i', 'testsrc=duration=5:size=3840x2160:rate=30',
    '-c:v', 'libx264', '-pix_fmt', 'yuv420p', 'mock_4k.mp4'
], capture_output=True, text=True)

if result.returncode == 0:
    size = os.path.getsize("mock_4k.mp4")
    print(f"✅ Success! File Size: {size / 1024 / 1024:.2f} MB")
else:
    print("❌ GENERATION FAILED!")
    print("--- FFMPEG ERROR LOG ---")
    print(result.stderr)
    print("------------------------")

🎥 Generating Mock 4K Source (3840x2160)...
✅ Success! File Size: 0.31 MB


In [5]:
# BLOCK 4: RUN PROCESSOR
print("🚀 Starting Dynamic Processing...\n")
print("--- SCRIPT OUTPUT START ---")

processor = ABRVideoProcessor()
output_root = "./public_videos"

try:
    processor.process('mock_4k.mp4', output_root)
except Exception as e:
    print(f"❌ Error: {e}")

print("--- SCRIPT OUTPUT END ---")

🚀 Starting Dynamic Processing...

--- SCRIPT OUTPUT START ---
{"status": "success", "stream_url": "/videos/mock_4k/master.m3u8", "poster_url": "/videos/mock_4k/poster.jpg"}
--- SCRIPT OUTPUT END ---


In [6]:
# BLOCK 5: VERIFY OUTPUT FOLDERS
print("\n🔍 Verifying Output Structure:")
base_path = os.path.join(output_root, "mock_4k")

# Expected: 8k should be MISSING. Others should exist.
checklist = [
    {"name": "8k",    "expect": False},
    {"name": "4k",    "expect": True},
    {"name": "1440p", "expect": True},
    {"name": "1080p", "expect": True},
    {"name": "720p",  "expect": True}
]

for item in checklist:
    path = os.path.join(base_path, item["name"])
    exists = os.path.exists(path)

    if item["expect"] == exists:
        status = "✅ PASS"
    else:
        status = "❌ FAIL"

    print(f"{status} | {item['name'].ljust(6)} | {'Created' if exists else 'Skipped'}")


🔍 Verifying Output Structure:
✅ PASS | 8k     | Skipped
✅ PASS | 4k     | Created
✅ PASS | 1440p  | Created
✅ PASS | 1080p  | Created
✅ PASS | 720p   | Created


In [7]:
# BLOCK 6: PROBE THE CHUNKS
print("\n🔬 Deep Probe of Generated Files:")

# Let's check the 1080p version to ensure it was actually downscaled
test_chunk = f"{output_root}/mock_4k/1080p/seg_000.ts"

if os.path.exists(test_chunk):
    try:
        probe = ffmpeg.probe(test_chunk)
        video_stream = next(s for s in probe['streams'] if s['codec_type'] == 'video')
        w = video_stream['width']
        h = video_stream['height']

        print(f"File: 1080p/seg_000.ts")
        print(f"Actual Resolution: {w}x{h}")

        if w == 1920 and h == 1080:
             print("✅ SCALING SUCCESSFUL")
        else:
             print("❌ SCALING FAILED")

    except Exception as e:
        print(f"Probe Error: {e}")
else:
    print("❌ 1080p Chunk not found to probe.")


🔬 Deep Probe of Generated Files:
File: 1080p/seg_000.ts
Actual Resolution: 1920x1080
✅ SCALING SUCCESSFUL
